In [28]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import clear_output
import time 
import os

In [29]:
def gallery(array, ncols=5):
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    assert nindex == nrows*ncols
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, intensity))
    return result

def make_array(knowledge,curr,cheat=False):
    index_i_size, index_j_size, index_k_size = knowledge.shape
    cow_agent_i, cow_agent_j= curr

    list = []
    for i in range(index_i_size):
        for j in range(index_j_size):
            image_name = "question"
          
            if(knowledge[i][j][VISITED]):
                image_name = "soil"
            if(knowledge[i][j][WET_SOIL] and knowledge[i][j][SMELLY]):
                image_name = "wet_soil_and_smelly"
            elif(knowledge[i][j][WET_SOIL]):
                image_name = "wet_soil"
            elif(knowledge[i][j][SMELLY]):
                image_name = "smelly"
            
            if(cheat):
                image_name = "soil"
            
            if(world[i][j][WOLF]):
                image_name = "wolf"
                
            if(world[i][j][QUICKSAND]):
                image_name = "mud"
                
            if(world[i][j][GRASS]):
                image_name = "grass"
                
            if(i==cow_agent_i and j==cow_agent_j):
                image_name = "cow"
                
            list.append(np.asarray(Image.open('images/'+image_name+'.jpg').resize((300, 300)).convert('RGB')))
    return np.array(list)



In [30]:
def get_adjacent(array,entity,value):
    index_i_size, index_j_size = default_world.shape
    
    if(entity[0]+1 < index_i_size and entity[0]+1 >= 0):
        array[entity[0]+1][entity[1]] = value
    
    if(entity[0]-1 < index_i_size and entity[0]-1 >= 0):
        array[entity[0]-1][entity[1]] = value

    if(entity[1]+1 < index_j_size and entity[1]+1 >= 0):
        array[entity[0]][entity[1]+1] = value
        
    if(entity[1]-1 < index_j_size and entity[1]-1 >= 0):
        array[entity[0]][entity[1]-1] = value
        
    return array

In [31]:
###### THIS BLOCK IS FOR THE VARIABLE ######
### CONSTANT FOR OUR PROPOSITION
QUICKSAND = 0
WET_SOIL  = 1
WOLF      = 2
SMELLY    = 3
GRASS     = 4
VISITED   = 5


## THIS BLOCK IS FOR THE VARIABLE
world     = np.zeros([ 5, 5 , 6]) 
knowledge = np.zeros([5, 5, 6 ]) 

grass = [
    [0,4],
    [2,2],
    [4,4],
    [3,0],
    [4,2]
]
# THIS ARRAY IS FOR GETTING ADJACENT BLOCK
adjacent_i = [0, 1, 0, -1]
adjacent_j = [-1, 0, 1, 0]

# THIS VARIABLE FOR THE GRID ENVIRONMENT
grid = np.array([
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11,12,13,14,15],
        [16,17,18,19,20],
        [21,22,23,24,25],
        ])


diagonal = [
        # LEFT to Up
        [[0,-1],[-1,0]],
        # Up to Right
        [[-1,0],[0,1]],
        # Right to Down
        [[0,1],[1,0]],
        # Right to Down
        [[1,0],[0,-1]]
    ]


In [32]:
def get_adjacent(array,entity,CONSTANT):
    index_i, index_j = entity
    index_i_size, index_j_size, index_k_size = array.shape
    
    if(index_i + 1 < index_i_size):
        array[index_i+1][index_j][CONSTANT] = True
    
    if(index_i - 1  >= 0):
        array[index_i-1][index_j][CONSTANT] = True

    if(index_j + 1 < index_j_size):
        array[index_i][index_j+1][CONSTANT] = True
        
    if(index_j - 1 >= 0):
        array[index_i][index_j-1][CONSTANT] = True

    return array

def initialize_world():
    world = np.zeros([ 5, 5 , 6]) 
    wolf_position = (1,1)
    wolf_i, wolf_j = wolf_position
    world[wolf_i][wolf_j][WOLF] = True
    world = get_adjacent(world,wolf_position,SMELLY)

    quicksand     = (3,2)
    quicksand_i, quicksand_j = quicksand
    world[quicksand_i][quicksand_j][QUICKSAND] = True
    world = get_adjacent(world,quicksand,WET_SOIL)

    for i in range(5):
        world[grass[i][0]][grass[i][1]][GRASS] = True
        
    return world

In [33]:
def agent_inferrence(agent,CONSTANT):
    global cow_agent

    index_i, index_j= agent

    # RULE 1 : 
    # If an adjacent is visited and theres no traces of bad percepts return safe
    for x in range(4):
        i = index_i + adjacent_i[x]
        j = index_j + adjacent_j[x]
        if(isValidRange((i,j))):
            if(cow_agent.knowledge[i][j][VISITED] and not cow_agent.knowledge[i][j][CONSTANT]):
                return False
          
    # RULE 2 : 
    # If an adjacent is visited and theres no traces of bad percepts return safe
    for x in range(4):
        point_a_i = index_i + diagonal[x][0][0]
        point_a_j = index_j + diagonal[x][0][1]
        
        point_b_i = index_i + diagonal[x][1][0]
        point_b_j = index_j + diagonal[x][1][1]
        
        point_c_i = index_i + (diagonal[x][0][0] + diagonal[x][1][0])
        point_c_j = index_j + (diagonal[x][1][0] + diagonal[x][1][1])
        
        if(isValidRange((point_c_i,point_c_j))):
            if(cow_agent.knowledge[point_c_i][point_c_j][VISITED]):
                return True
            else:
                count = 0
                for x in range(4):
                    point_c_adj_i = point_c_i + adjacent_i[x]
                    point_c_adj_j = point_c_j + adjacent_j[x]
                    if(isValidRange((point_c_adj_i,point_c_adj_j))):
                        if(cow_agent.knowledge[point_c_adj_i][point_c_adj_j][CONSTANT]):
                            count += 1

                if(count>=4):
                    return False

    # RULE 3 : 
    # If an adjacent is visited and theres no traces of bad percepts return safe
    for x in range(4):
        i = index_i + adjacent_i[x]
        j = index_j + adjacent_j[x]
        if(isValidRange((i,j))):
            if(cow_agent.knowledge[i][j][CONSTANT]):
                position = (i,j)
                return inferrence(position,CONSTANT)
            
def isValidRange(position):
    index_i,index_j = position
    if (index_i < 0 or index_j < 0 or index_i >= index_i_size or index_j >= index_j_size):
        return False
    
    return True
    
def isValid(row, col):
    global index_i_size
    global index_j_size
    global cow_agent
    
    if (row < 0 or col < 0 or row >= index_i_size or col >= index_j_size):
        return False
 
    if (cow_agent.knowledge[row][col][VISITED]):
        return False
    
    if(agent_inferrence((row, col),WET_SOIL)):
        return False
    
    if(agent_inferrence((row, col),SMELLY)):
        return False
    
    return True

def inferrence(position,CONSTANT):
    index_i,index_j = position
    
    for x in range(4):
        adj_i = index_i + adjacent_i[x]
        adj_j = index_j + adjacent_j[x]

        count = 0
        for y in range(4):
            i = adj_i + adjacent_i[y]
            j = adj_j + adjacent_j[y]
            if(isValidRange((i,j))):
                if(cow_agent.knowledge[i][j][CONSTANT]):
                    count += 1

        if(count>=2):
            return False

    return True

In [34]:
def grid_bfs(starting_point , grid):
    global cow_agent
    row,col = starting_point
    queue = []
    queue.append([row, col])
    while (len(queue) > 0):
        current_point = queue[0]
        queue.remove(queue[0])
        row = current_point[0]
        col = current_point[1]
        
        if (isValid(row, col) == False):
            continue
            
        cow_agent.perceive(world,current_point)
        cow_agent.knowledge[row][col][VISITED] = True
        clear_output(wait=True)
        array = make_array(cow_agent.knowledge,current_point)
        result = gallery(array)
        plt.figure(figsize = (12,12))
        plt.imshow(result)
        plt.axis('off')
        plt.show()
        if(len(cow_agent.grass_eatean)>=3):
            print("Goal Achieved Congratulations !!")
            break

        
        for i in range(4):
                index_i = row + adjacent_i[i]
                index_j = col + adjacent_j[i]
                if(isValidRange((index_i,index_j))):
                    queue.append([index_i, index_j])

In [35]:
def grid_dfs( starting_point , grid):
    global visited
    global cow_agent
    row,col = starting_point
    stack = []
    stack.append([row, col])
    while (len(stack) > 0):
        current_point = stack[len(stack) - 1]
        stack.remove(stack[len(stack) - 1])
        row = current_point[0]
        col = current_point[1]
        
        if (isValid(row, col) == False):
            continue
            
        cow_agent.perceive(world,current_point)
        cow_agent.knowledge[row][col][VISITED] = True
        clear_output(wait=True)
        array = make_array(cow_agent.knowledge,current_point)
        result = gallery(array)
        plt.figure(figsize = (12,12))
        plt.imshow(result)
        plt.axis('off')
        plt.show()
        
        if(len(cow_agent.grass_eatean)>=3):
            print("Goal Achieved Congratulations !!")
            break
        print("PROCESS: STACKING .........")
        
        
        for i in range(4):
                index_i = row + adjacent_i[i]
                index_j = col + adjacent_j[i]
                if(isValidRange((index_i,index_j))):
                    stack.append([index_i, index_j])




In [36]:
class Cow:
    def __init__(self,initial_position = (0,0)):
        self.knowledge =  np.zeros([5, 5, 6 ])
        self.grass_eatean = []
        self.initial_position = initial_position
        index_i, index_j = initial_position
        
    def perceive(self,world,point):
        index_i, index_j = point
        self.knowledge[index_i][index_j] = world[index_i][index_j] 
        
        if(world[index_i][index_j][GRASS]):
            world[index_i][index_j][GRASS] = False
            self.grass_eatean.append(point)
            
    def is_visited(self,point):
        index_i, index_j = point
        
        if(self.knowledge[index_i][index_j][VISITED]):
            return True
        
        return False

    def down(self,point):
        index_i, index_j = point
        return (index_i+1,index_j)

    def up(self,point):
        index_i, index_j = point
        return (index_i-i,index_j)

    def right(self,point):
        index_i, index_j = point
        return (index_i,index_j+1)

    def left(self,point):
        index_i, index_j = point
        return (index_i,index_j-1)
    
    def simulate(self):
        queue = self.grass_eatean
        print(queue[0])

In [ ]:
user_input = ""
number = [[0,0],[0,1],[0,2],[0,3],[0,4]]

while user_input.lower() != "exit":
    print("Welcome to the Cow World !")
    print(" Enter '1' for BFS ")
    print(" Enter '2' for DFS ")
    print(" Enter '3' for Show World ")
    print(" Enter 'Exit' to exit ")
    user_input = input('Please enter mode: ')
    if( user_input.lower()=="bfs" or user_input=="1" ):
        user_input = input('Enter State number between 1-5: ')
        try:
            val = int(user_input)
            if(val<0 or val>5):
                print("Enter valid range. Please try again")
                continue
        except ValueError:
            print("That's not an int. Please try again")
            continue
        index_i_size, index_j_size = grid.shape
        starting_point = number[val-1]
        cow_agent = Cow(starting_point)
        world = initialize_world()
        grid_bfs( starting_point, grid)
        input('Press Enter to continue')
        clear_output(wait=True)
    elif( user_input.lower()=="dfs" or user_input=="2" ):
        user_input = input('Enter State number between 1-5: ')
        try:
            val = int(user_input)
            if(val<0 or val>5):
                print("Enter valid range. Please try again")
                continue
        except ValueError:
            print("That's not an int. Please try again")
            continue
        index_i_size, index_j_size = grid.shape
        starting_point = number[val-1]
        cow_agent = Cow()
        world = initialize_world()
        grid_dfs( starting_point , grid)
        input('Press Enter to continue')
    elif( user_input.lower()=="cheat" or user_input=="3" ):
        world = initialize_world()
        starting_point = (0, 0)
        array = make_array(world,(0,0),cheat=True)
        result = gallery(array)
        plt.figure(figsize = (12,12))
        plt.imshow(result)
        plt.axis('off')
        plt.show()
        input('Press Enter to continue')
        clear_output(wait=True)

    clear_output(wait=True)
    
    


Welcome to the Cow World !
 Enter '1' for BFS 
 Enter '2' for DFS 
 Enter '3' for Show World 
 Enter 'Exit' to exit 
